## Assignment 3: $k$ Nearest Neighbor

**Do two questions.**

`! git clone https://github.com/DS3001/knn`

In [1]:
! git clone https://github.com/DS3001/knn

Cloning into 'knn'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 40 (delta 12), reused 12 (delta 5), pack-reused 17
Receiving objects: 100% (40/40), 11.00 MiB | 12.78 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

**Q1.** This question is a case study for $k$ nearest neighbor The target variable `y` is `price` and the features are `year` and `mileage`.

1. Load the `./data/USA_cars_datasets.csv`. Keep the following variables and drop the rest: `price`, `year`, `mileage`. Are there any `NA`'s to handle? Look at the head and dimensions of the data.
2. Maxmin normalize `year` and `mileage`.
3. Split the sample into ~80% for training and ~20% for evaluation.
4. Use the $k$NN algorithm and the training data to predict `price` using `year` and `mileage` for the test set for $k=3,10,25,50,100,300$. For each value of $k$, compute the mean squared error and print a scatterplot showing the test value plotted against the predicted value. What patterns do you notice as you increase $k$?
5. Determine the optimal $k$ for these data.
6. Describe what happened in the plots of predicted versus actual prices as $k$ varied, taking your answer into part 6 into account. (Hint: Use the words "underfitting" and "overfitting".)

**Q2.** This question is a case study for $k$ nearest neighbor. The data for the question include:

- age: age of the patient (years)
- anaemia: decrease of red blood cells or hemoglobin (boolean)
- high blood pressure: if the patient has hypertension (boolean)
- creatinine phosphokinase (CPK): level of the CPK enzyme in the blood (mcg/L)
- diabetes: if the patient has diabetes (boolean)
- ejection fraction: percentage of blood leaving the heart at each contraction (percentage)
- platelets: platelets in the blood (kiloplatelets/mL)
- sex: woman or man (binary)
- serum creatinine: level of serum creatinine in the blood (mg/dL)
- serum sodium: level of serum sodium in the blood (mEq/L)
- smoking: if the patient smokes or not (boolean)
- time: follow-up period (days)
- death event: if the patient deceased during the follow-up period (boolean)

1. Load the `./data/heart_failure_clinical_records_dataset.csv`. Are there any `NA`'s to handle? use `.drop()` to remove `time` from the dataframe.
2. Make a correlation matrix. What variables are strongly associated with a death event?
3. For the dummy variables `anaemia`, `diabetes`, `high_blood_pressure`, `sex`, and `smoking`, compute a summary table of `DEATH_EVENT` grouped by the variable. For which variables does a higher proportion of the population die when the variable takes the value 1 rather than 0?
4. On the basis of your answers from 2 and 3, build a matrix $X$ of the variables you think are most predictive of a death, and a variable $y$ equal to `DEATH_EVENT`.
5. Maxmin normalize all of the variables in `X`.
6. Split the sample into ~80% for training and ~20% for evaluation. (Try to use the same train/test split for the whole question, so that you're comparing apples to apples in the questions below.).
7. Determine the optimal number of neighbors for a $k$NN regression for the variables you selected.
8. OK, do steps 5 through 7 again, but use all of the variables (except `time`). Which model has a lower Sum of Squared Error? Which would you prefer to use in practice, if you had to predict `DEATH_EVENT`s? If you play with the selection of variables, how much does the SSE change for your fitted model on the test data? Are more variables always better? Explain your findings.

In [9]:
df = pd.read_csv("./knn/data/heart_failure_clinical_records_dataset.csv")

df = df.drop(columns=["time"])
df.isna().sum()
# There does not appear to be any NAs in this dataset

age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
DEATH_EVENT                 0
dtype: int64

In [13]:
# Reordering columns so 'DEATH_EVENT' appears first
df = df.rename(columns={'DEATH_EVENT':'death_event'})
df = df[['death_event', 'age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking']]

correlation_matrix = df.corr()
correlation_matrix
# Age, ejection fraction, serum creatinine, and serum sodium are the most strongly
# correlated with death event.

Index(['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'DEATH_EVENT'],
      dtype='object')


,death_event,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking
death_event,1.000000,0.253729,0.066270,0.062728,-0.001943,-0.268603,0.079351,-0.049139,0.294278,-0.195204,-0.004316,-0.012623
age,0.253729,1.000000,0.088006,-0.081584,-0.101012,0.060098,0.093289,-0.052354,0.159187,-0.045966,0.065430,0.018668
anaemia,0.066270,0.088006,1.000000,-0.190741,-0.012729,0.031557,0.038182,-0.043786,0.052174,0.041882,-0.094769,-0.107290
creatinine_phosphokinase,0.062728,-0.081584,-0.190741,1.000000,-0.009639,-0.044080,-0.070590,0.024463,-0.016408,0.059550,0.079791,0.002421
diabetes,-0.001943,-0.101012,-0.012729,-0.009639,1.000000,-0.004850,-0.012732,0.092193,-0.046975,-0.089551,-0.157730,-0.147173
ejection_fraction,-0.268603,0.060098,0.031557,-0.044080,-0.004850,1.000000,0.024445,0.072177,-0.011302,0.175902,-0.148386,-0.067315
high_blood_pressure,0.079351,0.093289,0.038182,-0.070590,-0.012732,0.024445,1.000000,0.049963,-0.004935,0.037109,-0.104615,-0.055711
platelets,-0.049139,-0.052354,-0.043786,0.024463,0.092193,0.072177,0.049963,1.000000,-0.041198,0.062125,-0.125120,0.028234
serum_creatinine,0.294278,0.159187,0.052174,-0.016408,-0.046975,-0.011302,-0.004935,-0.041198,1.000000,-0.189095,0.006970,-0.027414
serum_sodium,-0.195204,-0.045966,0.041882,0.059550,-0.089551,0.175902,0.037109,0.062125,-0.189095,1.000000,-0.027566,0.004813


In [20]:
vars = ['anaemia','diabetes','high_blood_pressure','sex','smoking','death_event']
group = ['anaemia','diabetes','high_blood_pressure','sex','smoking']
df.loc[:, vars].groupby(group).describe()

anaemia diabetes high_blood_pressure sex smoking death_event            \
                                                          count      mean   
0        0        0                   0   0       0        15.0  0.333333   
1        0        0                   0   1       0        20.0  0.400000   
2        0        0                   0   1       1        32.0  0.187500   
3        0        0                   1   0       0        10.0  0.200000   
4        0        0                   1   0       1         1.0  0.000000   
5        0        0                   1   1       0        12.0  0.333333   
6        0        0                   1   1       1         8.0  0.375000   
7        0        1                   0   0       0        17.0  0.176471   
8        0        1                   0   1       0        17.0  0.235294   
9        0        1                   0   1       1        12.0  0.333333   
10       0        1                   1   0       0         9.0  0.333333   
11       0        1                   1   0       1         1.0  1.000000   
12       0        1                   1   1       0         8.0  0.375000   
13       0        1                   1   1       1         8.0  0.500000   
14       1        0                   0   0       0        13.0  0.153846   
15       1        0                   0   1       0        17.0  0.588235   
16       1        0                   0   1       1        15.0  0.200000   
17       1        0                   1   0       0        10.0  0.400000   
18       1        0                   1   0       1         1.0  1.000000   
19       1        0                   1   1       0        11.0  0.272727   
20       1        0                   1   1       1         9.0  0.555556   
21       1        1                   0   0       0        15.0  0.400000   
22       1        1                   0   0       1         1.0  1.000000   
23       1        1                   0   1       0        14.0  0.214286   
24       1        1                   0   1       1         6.0  0.333333   
25       1        1                   1   0       0        12.0  0.500000   
26       1        1                   1   1       0         3.0  0.000000   
27       1        1                   1   1       1         2.0  0.000000   

                                        
         std  min  25%  50%   75%  max  
0   0.487950  0.0  0.0  0.0  1.00  1.0  
1   0.502625  0.0  0.0  0.0  1.00  1.0  
2   0.396558  0.0  0.0  0.0  0.00  1.0  
3   0.421637  0.0  0.0  0.0  0.00  1.0  
4        NaN  0.0  0.0  0.0  0.00  0.0  
5   0.492366  0.0  0.0  0.0  1.00  1.0  
6   0.517549  0.0  0.0  0.0  1.00  1.0  
7   0.392953  0.0  0.0  0.0  0.00  1.0  
8   0.437237  0.0  0.0  0.0  0.00  1.0  
9   0.492366  0.0  0.0  0.0  1.00  1.0  
10  0.500000  0.0  0.0  0.0  1.00  1.0  
11       NaN  1.0  1.0  1.0  1.00  1.0  
12  0.517549  0.0  0.0  0.0  1.00  1.0  
13  0.534522  0.0  0.0  0.5  1.00  1.0  
14  0.375534  0.0  0.0  0.0  0.00  1.0  
15  0.507300  0.0  0.0  1.0  1.00  1.0  
16  0.414039  0.0  0.0  0.0  0.00  1.0  
17  0.516398  0.0  0.0  0.0  1.00  1.0  
18       NaN  1.0  1.0  1.0  1.00  1.0  
19  0.467099  0.0  0.0  0.0  0.50  1.0  
20  0.527046  0.0  0.0  1.0  1.00  1.0  
21  0.507093  0.0  0.0  0.0  1.00  1.0  
22       NaN  1.0  1.0  1.0  1.00  1.0  
23  0.425815  0.0  0.0  0.0  0.00  1.0  
24  0.516398  0.0  0.0  0.0  0.75  1.0  
25  0.522233  0.0  0.0  0.5  1.00  1.0  
26  0.000000  0.0  0.0  0.0  0.00  0.0  
27  0.000000  0.0  0.0  0.0  0.00  0.0

**Q3.** Let's do some very basic computer vision. We're going to import the MNIST handwritten digits data and $k$NN to predict values (i.e. "see/read").

1. To load the data, run the following code in a chunk:
```
from keras.datasets import mnist
df = mnist.load_data('minst.db')
train,test = df
X_train, y_train = train
X_test, y_test = test
```
The `y_test` and `y_train` vectors, for each index `i`, tell you want number is written in the corresponding index in `X_train[i]` and `X_test[i]`. The value of `X_train[i]` and `X_test[i]`, however, is a 28$\times$28 array whose entries contain values between 0 and 256. Each element of the matrix is essentially a "pixel" and the matrix encodes a representation of a number. To visualize this, run the following code to see the first ten numbers:
```
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000)
for i in range(5):
    print(y_test[i],'\n') # Print the label
    print(X_test[i],'\n') # Print the matrix of values
    plt.contourf(np.rot90(X_test[i].transpose())) # Make a contour plot of the matrix values
    plt.show()
```
OK, those are the data: Labels attached to handwritten digits encoded as a matrix.

2. What is the shape of `X_train` and `X_test`? What is the shape of `X_train[i]` and `X_test[i]` for each index `i`? What is the shape of `y_train` and `y_test`?
3. Use Numpy's `.reshape()` method to covert the training and testing data from a matrix into an vector of features. So, `X_test[index].reshape((1,784))` will convert the $index$-th element of `X_test` into a $28\times 28=784$-length row vector of values, rather than a matrix. Turn `X_train` into an $N \times 784$ matrix $X$ that is suitable for scikit-learn's kNN classifier where $N$ is the number of observations and $784=28*28$ (you could use, for example, a `for` loop).
4. Use the reshaped `X_test` and `y_test` data to create a $k$-nearest neighbor classifier of digit. What is the optimal number of neighbors $k$? If you can't determine this, play around with different values of $k$ for your classifier.
5. For the optimal number of neighbors, how well does your predictor perform on the test set?
6. So, this is how computers "see." They convert an image into a matrix of values, that matrix becomes a vector in a dataset, and then we deploy ML tools on it as if it was any other kind of tabular data. To make sure you follow this, invent a way to represent a color photo in matrix form, and then describe how you could convert it into tabular data. (Hint: RGB color codes provide a method of encoding a numeric value that represents a color.)